# Music VAE

Recurrent VAE + Novel Hierarchical Decoder

VAE : Input $x$ 를 잘 설명하는 Feature 를 추출하여 Latent Vector(Representation) $z$ 에 담은 뒤, 이 Latent Vector $z$ 를 통해 $x$ 와 유사하지만 새로운 데이터를 생성하는 것을 목표로 하는 AutoEncoder 의 변형

Recurrent VAE : VAE 의 Encoder 와 Decoder 로 RNN 을 적용 -> long-term dependencies 문제로 LSTM 으로 교체

Music VAE : 여전한 Long sequence 에 대한 성능 저하를 Novel Hierarchical Decoder 구조를 사용하여 해결할 수 있는 Network

# Dependencies

- python3.8
- pip==21.1.3
- setuptools==57.4.0
- magenta==2.1.0

In [19]:
!venv/bin/pip install magenta==2.1.0
!venv/bin/pip install tensorflow-gpu

  Using cached magenta-2.1.0-py3-none-any.whl (1.4 MB)
  Using cached pygtrie-2.5.0-py3-none-any.whl (25 kB)
  Using cached python-rtmidi-1.1.2.tar.gz (204 kB)
  Using cached mir_eval-0.7-py3-none-any.whl
  Using cached dm_sonnet-2.0.0-py3-none-any.whl (254 kB)
  Using cached imageio-2.22.4-py3-none-any.whl (3.4 MB)
  Using cached note_seq-0.0.5-py3-none-any.whl (209 kB)
  Using cached note_seq-0.0.4-py3-none-any.whl (205 kB)
  Using cached note_seq-0.0.3-py3-none-any.whl (210 kB)
  Created wheel for python-rtmidi: filename=python_rtmidi-1.1.2-cp38-cp38-linux_x86_64.whl size=422310 sha256=fd34760822157cfb084c079543e743021ad1083b823bb9058cae8e19db40fd3c
  Stored in directory: /home/heerak/.cache/pip/wheels/23/93/e9/7d805b982c4cb5c6cec3e77e1fc6e7417a193beca2230cea52
Successfully built python-rtmidi
You should consider upgrading via the '/data/heerak/music_vae/venv/bin/python -m pip install --upgrade pip' command.


# Import

In [11]:
import os
import numpy as np
import collections

import tensorflow.compat.v1 as tf
import tf_slim
import note_seq

from magenta.common import merge_hparams
from magenta.contrib import training as contrib_training
from magenta.models.music_vae import MusicVAE, lstm_models, data, configs
from magenta.scripts.convert_dir_to_note_sequences import convert_directory
from magenta.models.music_vae.trained_model import TrainedModel

# 전처리 진행하기 위한 과정

- https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip 다운로드
- zip 파일 압축해제
- MIDI 데이터를 NoteSequences 로 변환하고 TFRecord 파일로 변환해주는 <br>
  magenta 의 scripts/convert_dir_to_note_sequences.py 에 있는 convert_directory function 이용해 Directory 채로 전처리 진행

## zip 파일 다운로드

In [2]:
root_path = os.getcwd()

In [30]:
url = "https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip"
# 파일 다운로드
tf.keras.utils.get_file(origin=url,
                        fname=os.path.join(root_path, 'data.zip'),
                        extract=True)

3260318/3260318 [==============================] - 0s 0us/step


'/data/heerak/music_vae/data.zip'

## zip 파일 압축해제

In [31]:
# 압축해제
!unzip data.zip

Archive:  data.zip
   creating: groove/
   creating: groove/drummer8/
   creating: groove/drummer8/session2/
  inflating: groove/drummer8/session2/12_funk_81_beat_4-4.mid  
  inflating: groove/drummer8/session2/25_latin_84_beat_4-4.mid  
  inflating: groove/drummer8/session2/2_funk_92_beat_4-4.mid  
  inflating: groove/drummer8/session2/41_highlife_126_beat_4-4.mid  
  inflating: groove/drummer8/session2/40_rock-indie_104_beat_4-4.mid  
  inflating: groove/drummer8/session2/33_rock_117_beat_4-4.mid  
  inflating: groove/drummer8/session2/30_afrobeat_98_beat_4-4.mid  
  inflating: groove/drummer8/session2/39_rock-indie_104_beat_4-4.mid  
  inflating: groove/drummer8/session2/14_afrobeat_103_beat_4-4.mid  
  inflating: groove/drummer8/session2/17_afrobeat_85_beat_4-4.mid  
  inflating: groove/drummer8/session2/31_afrobeat_98_beat_4-4.mid  
  inflating: groove/drummer8/session2/38_rock_112_beat_4-4.mid  
  inflating: groove/drummer8/session2/5_funk_91_beat_4-4.mid  
  inflating: groove/dr

## MIDI 데이터를 NoteSequences 로 변환하고 TFRecord 파일로 저장 

In [34]:
# 폴더내에 MIDI 파일을 찾아서 NoteSequences 로 변환하고 TFRecord 로 저장해주는 function 이용
if not os.path.exists(os.path.join(root_path, 'tfrecord')):
    os.makedirs(os.path.join(root_path, 'tfrecord'))
convert_directory(
    root_dir=os.path.join(root_path, 'groove'),  # MIDI 파일 저장되어 있는 전체 데이터 폴더
    output_file=os.path.join(root_path, 'tfrecord/groove.tfrecord'),  # tfrecord 파일 저장 위치
    recursive=True,  # 재귀적으로 폴더 내에 모든 위치 찾도록 지정해주는 parameter
)

INFO:tensorflow:Converting files in '/data/heerak/music_vae/groove/'.


INFO:tensorflow:Converting files in '/data/heerak/music_vae/groove/'.


INFO:tensorflow:0 files converted.


INFO:tensorflow:0 files converted.


INFO:tensorflow:Converting files in '/data/heerak/music_vae/groove/drummer3'.


...

INFO:tensorflow:Converted MIDI file /data/heerak/music_vae/groove/drummer4/session1/2_jazz_158_beat_4-4.mid.


INFO:tensorflow:Converted MIDI file /data/heerak/music_vae/groove/drummer4/session1/8_blues-shuffle_112_beat_4-4.mid.


INFO:tensorflow:Converted MIDI file /data/heerak/music_vae/groove/drummer4/session1/8_blues-shuffle_112_beat_4-4.mid.


INFO:tensorflow:Converted MIDI file /data/heerak/music_vae/groove/drummer4/session1/4_latin-brazilian_92_beat_4-4.mid.


INFO:tensorflow:Converted MIDI file /data/heerak/music_vae/groove/drummer4/session1/4_latin-brazilian_92_beat_4-4.mid.


# 모델 구조

<img src="archive/music_VAE.jpg" width="500" height="340">

## Encoder -> two-layer bidirectional LSTM Network
- hidden state size 2048
- Latent Dimensions 512

## Conductor -> two-layer unidirectional LSTM Network
- hidden state size 1024
- output dimension 512

## Decoder -> two-layer unidirectional LSTM
    - hidden state size 1024

# Model Config

In [3]:
class Config(collections.namedtuple(
    'Config', 
    ['model', 'hparams', 'note_sequence_augmenter', 'data_converter',
     'train_examples_path', 'eval_examples_path', 'tfds_name'])):
    
    def values(self):
        return self._asdict()
    
    
CONFIG_MAP = {}

HParams = contrib_training.HParams

# groovae_4bar 모델 구조 사용
CONFIG_MAP['groovae_4bar'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(),
                   lstm_models.GrooveLstmDecoder()),
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=128,  # 512 -> 128
            max_seq_len=16 * 4,
            z_size=512,
            enc_rnn_size=[2048, 2048],
            dec_rnn_size=[1024, 1024],
            max_beta=0.2,
            free_bits=48,
            dropout_keep_prob=0.3,
        )
    ),
    note_sequence_augmenter=None,
    data_converter=data.GrooveConverter(
        split_bars=4,
        steps_per_quarter=4,
        quarters_per_bar=4,
        max_tensors_per_notesequence=20,
        pitch_classes=data.REDUCED_DRUM_PITCH_CLASSES,
    ),
    train_examples_path=os.path.join(root_path, 'tfrecord', 'groove.tfrecord'),
    eval_examples_path=None,
    tfds_name=None,
)

# Train 에 필요한 function 정의

music_vae_train.py 에 들어있는 필요한 함수만 가져와서 사용

In [4]:
def _get_input_tensors(dataset, config):
    """Get input tensors from dataset."""
    batch_size = config.hparams.batch_size
    iterator = tf.data.make_one_shot_iterator(dataset)
    (input_sequence, output_sequence, control_sequence, sequence_length) = iterator.get_next()
    input_sequence.set_shape([batch_size, None, config.data_converter.input_depth])
    output_sequence.set_shape([batch_size, None, config.data_converter.output_depth])
    if not config.data_converter.control_depth:
        control_sequence = None
    else:
        control_sequence.set_shape([batch_size, None, config.data_converter.control_depth])
        sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())

    return {
        'input_sequence': input_sequence,
        'output_sequence': output_sequence,
        'control_sequence': control_sequence,
        'sequence_length': sequence_length
    }

In [5]:
def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep=5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):
    """Train loop."""
    tf.gfile.MakeDirs(train_dir)
    is_chief = (task == 0)

    with tf.Graph().as_default():
        with tf.device(tf.train.replica_device_setter(
        num_ps_tasks, merge_devices=True)):
            
            model = config.model
            model.build(config.hparams,
                        config.data_converter.output_depth,
                        is_training=True)
            
            optimizer = model.train(**_get_input_tensors(dataset_fn(), config))
            
            hooks = []
            if num_sync_workers:
                optimizer = tf.train.SyncReplicasOptimizer(
                    optimizer,
                    num_sync_workers)
                hooks.append(optimizer.make_session_run_hook(is_chief))
                
            grads, var_list = list(zip(*optimizer.compute_gradients(model.loss)))
            global_norm = tf.global_norm(grads)
            
            if config.hparams.clip_mode == 'value':
                g = config.hparams.grad_clip
                clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
            elif config.hparams.clip_mode == 'global_norm':
                clipped_grads = tf.cond(
                    global_norm < config.hparams.grad_norm_clip_to_zero,
                    lambda: tf.clip_by_global_norm(  # pylint:disable=g-long-lambda
                        grads, config.hparams.grad_clip, use_norm=global_norm)[0],
                    lambda: [tf.zeros(tf.shape(g)) for g in grads])
            else:
                raise ValueError(
                    'Unknown clip_mode: {}'.format(config.hparams.clip_mode))
                
            train_op = optimizer.apply_gradients(
                list(zip(clipped_grads, var_list)),
                global_step=model.global_step,
                name='train_step')
            
            logging_dict = {'global_step': model.global_step,
                            'loss': model.loss}
            
            hooks.append(tf.train.LoggingTensorHook(logging_dict, every_n_iter=100))
            if num_steps:
                hooks.append(tf.train.StopAtStepHook(last_step=num_steps))
                
            scaffold = tf.train.Scaffold(
                saver=tf.train.Saver(
                    max_to_keep=checkpoints_to_keep,
                    keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours))
            
            tf_slim.training.train(
                train_op=train_op,
                logdir=train_dir,
                scaffold=scaffold,
                hooks=hooks,
                save_checkpoint_secs=60,
                master=master,
                is_chief=is_chief)

In [6]:
def run(config_map,
        tf_file_reader=tf.data.TFRecordDataset,
        file_reader=tf.python_io.tf_record_iterator,
       ):
    
    """Load model params, save config file and start trainer.
    Args:
        config_map: Dictionary mapping configuration name to Config object.
        tf_file_reader: The tf.data.Dataset class to use for reading files.
        file_reader: The Python reader to use for reading files.
    Raises:
    ValueError: if required flags are missing or invalid.
    """
    
    config = config_map['groovae_4bar']
    train_dir = os.path.join(root_path, 'train')

    def dataset_fn():
        return data.get_dataset(
            config,
            tf_file_reader=tf_file_reader,
            is_training=True,
            cache_dataset=True)

    train(
        train_dir,
        config=config,
        dataset_fn=dataset_fn,
        num_steps=10000,  # 200000 -> 10000
    )

# Train

In [7]:
run(CONFIG_MAP)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 64, 'z_size': 512, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 128, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [1024, 1024], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 64, 'z_size': 512, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 128, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [1024, 1024], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048, 2048]



INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048, 2048]



INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]



...

INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 10000...


INFO:tensorflow:Saving checkpoints for 10000 into /data/heerak/music_vae/train/model.ckpt.


INFO:tensorflow:Saving checkpoints for 10000 into /data/heerak/music_vae/train/model.ckpt.


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 10000...


INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 10000...


# Generate

music_vae_generate.py 에서 필요한 부분만 정의하여 사용

In [17]:
def generate(
    config_map, 
    output_dir,
    
):
    tf.gfile.MakeDirs(os.path.join(root_path, output_dir))
    
    config = config_map['groovae_4bar']
  
    config.data_converter.max_tensors_per_item = None

    print('Loading model...')
    checkpoint_dir_or_path = os.path.join(root_path, 'train')

    model = TrainedModel(
        config, 
        batch_size=1,
        checkpoint_dir_or_path=checkpoint_dir_or_path
    )

    print('Sampling...')
    results = model.sample(
        n=1,
        length=16*4,
        temperature=0.5
    )

    note_seq.sequence_proto_to_midi_file(results[0], os.path.join(root_path, output_dir, 'drum_4bar.mid'))

    print('Done.')

In [19]:
# 사용하는 최신 checkpoint
tf.train.latest_checkpoint(os.path.join(root_path, 'train'))

'/data/heerak/music_vae/train/model.ckpt-10000'

In [20]:
generate(CONFIG_MAP, 'output')

Loading model...
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 64, 'z_size': 512, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 1, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [1024, 1024], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, GrooveLstmDecoder, and hparams:
{'max_seq_len': 64, 'z_size': 512, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 1, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [1024, 1024], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 0.3, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}


INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048, 2048]



INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048, 2048]



INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]



INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]



/data/heerak/music_vae/venv/lib/python3.8/site-packages/magenta/models/music_vae/lstm_utils.py:94: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tf.layers.dense(
/data/heerak/music_vae/venv/lib/python3.8/site-packages/magenta/contrib/rnn.py:750: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._kernel = self.add_variable(
/data/heerak/music_vae/venv/lib/python3.8/site-packages/magenta/contrib/rnn.py:752: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._bias = self.add_variable(


INFO:tensorflow:Restoring parameters from /data/heerak/music_vae/train/model.ckpt-10000


/data/heerak/music_vae/venv/lib/python3.8/site-packages/magenta/models/music_vae/base_model.py:195: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mu = tf.layers.dense(
/data/heerak/music_vae/venv/lib/python3.8/site-packages/magenta/models/music_vae/base_model.py:200: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  sigma = tf.layers.dense(
INFO:tensorflow:Restoring parameters from /data/heerak/music_vae/train/model.ckpt-10000


Sampling...
Done.
